In [1]:
from __future__ import print_function, division
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn

"""
define all the constants
"""
numEpochs = 10
seriesLength = 50000
backpropagationLength = 15
stateSize = 4
numClasses = 2
echoStep = 3
batchSize = 5
num_batches = seriesLength // batchSize // backpropagationLength


"""
generate data
"""
def generateData():
    x = np.array(np.random.choice(2, seriesLength, p=[0.5, 0.5]))
    y = np.roll(x, echoStep)
    y[0:echoStep] = 0

    x = x.reshape((batchSize, -1))
    y = y.reshape((batchSize, -1))

    return (x, y)


"""
start computational graph
"""
batchXHolder = tf.placeholder(tf.float32, [batchSize, backpropagationLength], name="x_input")
batchYHolder = tf.placeholder(tf.int32, [batchSize, backpropagationLength], name="y_input")

# rnn replace
#initState = tf.placeholder(tf.float32, [batchSize, stateSize], "rnn_init_state")

cellState = tf.placeholder(tf.float32, [batchSize, stateSize])
hiddenState = tf.placeholder(tf.float32, [batchSize, stateSize])
initState = rnn.LSTMStateTuple(cellState, hiddenState)

W = tf.Variable(np.random.rand(stateSize+1, stateSize), dtype=tf.float32, name="weight1")
bias1 = tf.Variable(np.zeros((1,stateSize)), dtype=tf.float32)

W2 = tf.Variable(np.random.rand(stateSize, numClasses),dtype=tf.float32, name="weight2")
bias2 = tf.Variable(np.zeros((1,numClasses)), dtype=tf.float32)

tf.summary.histogram(name="weights", values=W)


# Unpack columns
inputsSeries = tf.split(axis=1, num_or_size_splits=backpropagationLength, value=batchXHolder)
labelsSeries = tf.unstack(batchYHolder, axis=1)

# Forward passes

# rnn replace
# cell = rnn.BasicRNNCell(stateSize)
# statesSeries, currentState = rnn.static_rnn(cell, inputsSeries, initState)

cell = rnn.BasicLSTMCell(stateSize, state_is_tuple=True)
statesSeries, currentState = rnn.static_rnn(cell, inputsSeries, initState)

# calculate loss
logits_series = [tf.matmul(state, W2) + bias2 for state in statesSeries]
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits) for logits, labels in zip(logits_series,labelsSeries)]
total_loss = tf.reduce_mean(losses, name="total_loss")

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss, name="training")


"""
plot computation
"""
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batchSeriesIdx in range(5):
        oneHotOutputSeries = np.array(predictions_series)[:, batchSeriesIdx, :]
        singleOutputSeries = np.array([(1 if out[0] < 0.5 else 0) for out in oneHotOutputSeries])

        plt.subplot(2, 3, batchSeriesIdx + 2)
        plt.cla()
        plt.axis([0, backpropagationLength, 0, 2])
        left_offset = range(backpropagationLength)
        plt.bar(left_offset, batchX[batchSeriesIdx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batchSeriesIdx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, singleOutputSeries * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)


"""
run the graph
"""
with tf.Session() as sess:
    writer = tf.summary.FileWriter("logs", graph=tf.get_default_graph())
    sess.run(tf.global_variables_initializer())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(numEpochs):
        x,y = generateData()

        # rnn remove
        # _current_state = np.zeros((batchSize, stateSize))

        _current_cell_state = np.zeros((batchSize, stateSize))
        _current_hidden_state = np.zeros((batchSize, stateSize))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * backpropagationLength
            end_idx = start_idx + backpropagationLength

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, currentState, predictions_series],
                feed_dict={
                    batchXHolder:batchX,
                    batchYHolder:batchY,
                    cellState: _current_cell_state,
                    hiddenState: _current_hidden_state
                })

            _current_cell_state, _current_hidden_state = _current_state

            loss_list.append(_total_loss)

            # fix the cost summary later
            tf.summary.scalar(name="totalloss", tensor=_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


<Figure size 640x480 with 0 Axes>

New data, epoch 0
Step 0 Loss 0.69344735


<Figure size 640x480 with 6 Axes>

Step 100 Loss 0.5562319


...

Step 400 Loss 0.0010981325


<Figure size 640x480 with 6 Axes>

Step 500 Loss 0.0019068259


<Figure size 640x480 with 6 Axes>

KeyboardInterrupt: 

In [2]:
import numpy as np
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
np.random.permutation(np.arange(20))[:10]

array([ 6, 14,  2,  5, 19, 15,  9, 16, 18,  1])

In [19]:
sample_data = np.array([10,3,4,5,6,7])
sample_data[np.array([2,4])] = 0
sample_data

array([10,  3,  0,  5,  0,  7])

In [7]:
np.random.permutation(np.arange(3))[:2]

array([1, 0])

In [14]:
sample_data[np.random.permutation(np.arange(3))[:2]]= 0

IndexError: too many indices for array

In [13]:
np.array([2,4])

array([2, 4])

In [15]:
mask = np.zeros(10)
indices = np.random.permutation(np.arange(10))[:2]
mask[indices] = 1

In [18]:
mask[np.array([2,4])] = 100
print(mask)

[  0.   0. 100.   1. 100.   0.   0.   0.   0.   0.]


In [20]:
np.ones(10)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [21]:
signals = np.random.uniform(low=0.0, high=1.0, size=(10,100))

In [22]:
print(signals)

[[0.45324177 0.39524983 0.66662199 0.50177989 0.88715725 0.59072731
  0.5058612  0.91245671 0.05789386 0.9456987  0.5337271  0.46340839
  0.10770084 0.75555994 0.80571872 0.73744444 0.91039156 0.43462645
  0.48423017 0.13484443 0.95240136 0.71414801 0.19982145 0.4350095
  0.67460294 0.47264362 0.92360068 0.91862431 0.99873758 0.18765307
  0.05547406 0.37856128 0.38905897 0.80372172 0.13466623 0.52235018
  0.62078519 0.41700153 0.18860545 0.36424287 0.76518253 0.79549792
  0.40671591 0.65911169 0.50007392 0.17337722 0.92839697 0.54140628
  0.91461166 0.39021597 0.46729536 0.76714563 0.49514301 0.5373988
  0.09532923 0.72800938 0.85036218 0.49794856 0.89099277 0.27214015
  0.81174906 0.26698658 0.5632097  0.7333262  0.40407986 0.49445847
  0.93025618 0.53776366 0.66344353 0.53544005 0.97100923 0.59238622
  0.28314594 0.9465687  0.85864679 0.34161472 0.21179968 0.25955219
  0.49075958 0.26349919 0.71779037 0.74089657 0.15568176 0.23110257
  0.18727243 0.61685824 0.88908788 0.20414685 0.26

In [2]:
import numpy as np
np.arange(0,200)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18